In [ ]:
!pip install transformers datasets captum torch


This code shows the implementation of a distilled version of the BERT model. It is not as accurate as the full BERT however it takes a fraction of the time to run.

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score, classification_report
from google.colab import files

# Load the dataset from provided path
file_path = '/content/Sentences_with_Strong_Sentiment_Words (1).csv'
df = pd.read_csv(file_path)

# Display a preview of the dataset
print("Dataset Preview:")
print(df.head())


Dataset Preview:
     Word Sentiment                                           Sentence
0  accuse   Negatif  accuse is early and leads to disorders consequ...
1  accuse   Negatif  accuse is there and leads to corpse consequences.
2  accuse   Negatif   accuse is when and leads to insult consequences.
3  accuse   Negatif  accuse is on this note and leads to hunger con...
4  accuse   Negatif     accuse is well and leads to mess consequences.


In [ ]:
# Define special tokens to mark the target word
START_TOKEN = '[TARGET]'
END_TOKEN = '[/TARGET]'

# Add special tokens around the target word in each sentence
def mark_target_word(row):
    sentence = row['Sentence']
    word = row['Word']
    import re
    pattern = re.compile(re.escape(word), re.IGNORECASE)
    marked_sentence = pattern.sub(f"{START_TOKEN} {word} {END_TOKEN}", sentence, count=1)
    return marked_sentence

df['Marked_Sentence'] = df.apply(mark_target_word, axis=1)


In [ ]:
# Define label mapping
label_list = ['Negatif', 'Positif', 'Neutral']
label_map = {label: idx for idx, label in enumerate(label_list)}
df['Sentiment_Label'] = df['Sentiment'].map(label_map)

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize and align labels
def tokenize_data(row):
    encoding = tokenizer(row['Marked_Sentence'], padding='max_length', truncation=True, max_length=128)
    encoding['labels'] = label_map.get(row['Sentiment'], -100)
    return encoding

# Apply tokenization and convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_data, remove_columns=dataset.column_names)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [ ]:
# Define label mapping
label_list = ['Negatif', 'Positif', 'Neutral']
label_map = {label: idx for idx, label in enumerate(label_list)}
df['Sentiment_Label'] = df['Sentiment'].map(label_map)

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize and align labels
def tokenize_data(row):
    encoding = tokenizer(row['Marked_Sentence'], padding='max_length', truncation=True, max_length=128)
    encoding['labels'] = label_map.get(row['Sentiment'], -100)
    return encoding

# Apply tokenization and convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_data, remove_columns=dataset.column_names)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from captum.attr import IntegratedGradients
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [ ]:
# Define label mapping
label_list = ['Negatif', 'Positif', 'Neutral']
label_map = {label: idx for idx, label in enumerate(label_list)}
df['Sentiment_Label'] = df['Sentiment'].map(label_map)

# Initialize tokenizer for DistilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize and align labels
def tokenize_data(row):
    encoding = tokenizer(row['Marked_Sentence'], padding='max_length', truncation=True, max_length=64)
    encoding['labels'] = label_map.get(row['Sentiment'], -100)
    return encoding

# Apply tokenization and convert to Hugging Face Dataset format
dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize_data, remove_columns=dataset.column_names)
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1540 [00:00<?, ? examples/s]

In [ ]:
# Load DistilBERT model for multi-class classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_list))

# Define training arguments for quicker training
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,                  # Reduce to 1 epoch
    per_device_train_batch_size=4,       # Reduce batch size
    per_device_eval_batch_size=4,
    warmup_steps=10,                     # Fewer warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    max_steps=100                        # Limit steps for demonstration
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Evaluation metrics
def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1': f1_score(labels, preds, average='weighted')
    }


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


max_steps is given, it will override any value given in num_train_epochs


TrainOutput(global_step=100, training_loss=0.2139056968688965, metrics={'train_runtime': 283.5685, 'train_samples_per_second': 1.411, 'train_steps_per_second': 0.353, 'total_flos': 6623488051200.0, 'train_loss': 0.2139056968688965, 'epoch': 0.3246753246753247})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:", results)


Evaluation Results: {'eval_loss': 0.041546277701854706, 'eval_accuracy': 0.9935064935064936, 'eval_f1': 0.9902801600914807, 'eval_runtime': 34.6639, 'eval_samples_per_second': 8.885, 'eval_steps_per_second': 2.221, 'epoch': 0.3246753246753247}


In [ ]:
# Initialize Integrated Gradients for XAI
ig = IntegratedGradients(model)



In [ ]:
import shap
from transformers import pipeline
import torch

# Load a sentiment-analysis pipeline with BERT
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased")

def explain_with_shap(sentence, target_word):
    # Define the SHAP explainer using a Hugging Face pipeline
    explainer = shap.Explainer(sentiment_pipeline)

    # Run SHAP explanation on the sentence
    shap_values = explainer([sentence])

    # Display the SHAP values
    print(f"Sentence: {sentence}")
    shap.initjs()
    shap.plots.text(shap_values)

# Test with sample sentences
sample_sentences = [
    "accuse is early and leads to disorder consequences.",
    "accuse is when and leads to insult consequences.",
    "earth is well and leads to peaceful consequences.",
    "earth is in addition and leads to aggressive consequences."
]

for sentence in sample_sentences:
    target_word = 'accuse' if 'accuse' in sentence.lower() else 'earth'
    explain_with_shap(sentence, target_word)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/110 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:19, 19.01s/it]               

Sentence: accuse is early and leads to disorder consequences.


  0%|          | 0/110 [00:00<?, ?it/s]

...

  0%|          | 0/110 [00:00<?, ?it/s]

Sentence: earth is well and leads to peaceful consequences.


  0%|          | 0/132 [00:00<?, ?it/s]

PartitionExplainer explainer: 2it [00:12, 12.16s/it]               

Sentence: earth is in addition and leads to aggressive consequences.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 